# ---------------------------- Machine Learning - Metadados -------------------------------------

## O que é Metadados e para que serve ?
No contexto de Machine Learning podemos dizer que é a informação adicional aos dados que ajudam na tomada de decisão sobre quais tratamentos devem ser aplicados às variáveis
com a finalidade de obter a tabela de modelagem com melhor qualidade para treinamento de modelos.

Em resumo podemos dizer que á a informação essencial para o Data Prep !!

## Definindo uma ABT
Analytical Base Table ou Flat Table: trata-se da tabela que será utilizada para o treinamento dos modelos, nela devemos observar os seguintes campos(variáveis/features):

* Id: Chave de Cruzamento - Não deve apresentar duplicidades
* Inputs: Conjunto de variáveis explicativas (Numéricas e/ou Categóricas/String)
* Target: Variável resposta (Obrigatória para problemas supervisionados)



## Exemplo - Tabela de Modelagem 

In [3]:
# Tabela referente ao modelo Titanic disponibilizado no Kaggle(https://www.kaggle.com)
import pandas as pd
df00 = pd.read_csv('titanic_kaggle.csv')

In [4]:
df00.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# Vamos verificar as variáveis e os respectivos tipos
df00.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

#### Sabendo que:
* PassengerId: Id (Chave única)
* Survived: Target (binário)
* Demais: Input (Explicativas)

In [40]:
# Vamos renomear as variáveis PassengerId e Survived
df00.rename(columns={'PassengerId': 'id', 'Survived': 'target'}, inplace=True)

In [41]:
df00.head()

,id,target,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Exemplo de função que gera o metadados da tabela de modelagem

In [46]:
def AjusteMetadados(dataframe): 

    train = dataframe
    # Verifica os tipos de variáveis presentes na tabela de treino
    t = []
    for i in train.columns:
            t.append(train[i].dtype)

    n = []
    for i in train.columns:
            n.append(i)

    aux_t = pd.DataFrame(data=t,columns=["Tipos"])
    aux_n = pd.DataFrame(data=n,columns=["Features"])
    df_tipovars = pd.concat([aux_n, aux_t], axis=1, join_axes=[aux_n.index])

    data = []
    for f in train.columns:

        # Definindo o papel das variáveis:
        if f == 'target':
            role = 'target'
        elif f == 'id':
            role = 'id'
        else:
            role = 'input'

        # Definindo o tipo das variáveis: nominal, ordinal, binary ou interval
        if f == 'target':
            level = 'binary'
        if train[f].dtype == 'object' or f == 'id': 
            level = 'nominal'
        elif train[f].dtype in ['float','float64'] :
            level = 'interval'
        elif train[f].dtype in ['int','int64'] :
            level = 'ordinal'

        # Todas variáveis são incializadas com keep exceto o id
        keep = True
        if f == 'id':
            keep = False

        # Definindo o tipo das variáveis da tabela de entrada
        dtype = train[f].dtype

        # Criando a lista com todo metadados
        f_dict = {
            'Features': f,
            'Role': role,
            'Level': level,
            'Keep': keep,
            'Tipo': dtype
        }
        data.append(f_dict)

    meta = pd.DataFrame(data, columns=['Features', 'Role', 'Level', 'Keep', 'Tipo'])

    # Quantidade de domínios distintos para cada cariável do tipo ordinal e nominal
    card = []

    v = train.columns
    for f in v:
        dist_values = train[f].value_counts().shape[0]
        f_dict = {
                'Features': f,
                'Cardinality': dist_values
            }
        card.append(f_dict)

    card = pd.DataFrame(card, columns=['Features', 'Cardinality'])

    metadados_train = pd.merge(meta, card, on='Features')

    return metadados_train 

In [47]:
metadados = AjusteMetadados(df00)
metadados

,Features,Role,Level,Keep,Tipo,Cardinality
0,id,id,nominal,False,int64,891
1,target,target,ordinal,True,int64,2
2,Pclass,input,ordinal,True,int64,3
3,Name,input,nominal,True,object,891
4,Sex,input,nominal,True,object,2
5,Age,input,interval,True,float64,88
6,SibSp,input,ordinal,True,int64,7
7,Parch,input,ordinal,True,int64,7
8,Ticket,input,nominal,True,object,681
9,Fare,input,interval,True,float64,248


## Descrição sobre Level das variáveis
* Nominais: variáveis tipo string/char 
* Ordinais: variáveis numéricas ou char que podem ser interpretadas como numéricas com ordem (1<2<3<4...)
* Intervalares: variáveis numéricas de domínio ]-inf ; + inf [
* Binárias: variáveis numéricas que apresentam duas categorias [0;1]

Cardinalidade representa a quantidade de domínios distintos


### Salvando o metadados como csv
Após salvar metadados como csv verifique se o arquivo está no home do Jupyter e faça download do mesmo para seu PC

In [49]:
metadados.to_csv('metadados_aluno_n.csv', sep=',', encoding='utf-8',index=True)